In [1]:
import os
from PIL import Image, ImageSequence
import re
import copy


# Set directory containing the tiff files
image_dir = '/Volumes/SANDISK/images/wildLaby/2025-07-07_BF_4x_330umOverlap_1'
filename_prefix = '2025-07-07_BF_4x_330umOverlap_1_MMStack_2'

# Set the output directory where stitched images will be saved
output_dir = '/Users/johnwhitfield/Desktop/output'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set grid dimensions (rows and columns)
rows, cols = 7, 7

# Define overlap parameter (fraction of image size)
overlap = 0.1  # 6.7% overlap

# Define the number of timepoints to process (for faster debugging)
max_timepoints = 1000  # Set to the number of timepoints you'd like to process (e.g., 10)

# Extract the grid indices from the filename
def get_grid_indices(filename):
    match = re.search(r'Pos(\d{3})_(\d{3})', filename)
    if match:
        x_index = int(match.group(1))
        y_index = int(match.group(2))
        return x_index, y_index
    else:
        raise ValueError(f"Filename {filename} does not match expected pattern.")

# Load all image filenames
image_files = [f for f in os.listdir(image_dir) if f.endswith('.tif')]

# Initialize an empty dictionary to store image stacks with their grid positions
images_grid = {}

# Load each image stack and map to its grid position
for file in image_files:
    x_index, y_index = get_grid_indices(file)
    # Adjust for the inverted y-axis
    #y_index = rows - 1 - y_index
    # Load the tiff stack (multiple time points)
    image_path = os.path.join(image_dir, file)
    image_stack = Image.open(image_path)

    # Convert to 8-bit if necessary
    #if image_stack.mode in ['I;16', 'I']:
        #image_stack = image_stack.convert('L')  # Convert to 8-bit grayscale

    # Store in the dictionary using (x_index, y_index) as the key
    images_grid[(x_index, y_index)] = [copy.deepcopy(frame) for frame in ImageSequence.Iterator(image_stack)]  # Deep copy to ensure independent frames

# Determine the total number of time points (frames) from one of the stacks
num_timepoints = len(next(iter(images_grid.values())))

# Limit the number of timepoints to process for debugging
timepoints_to_process = min(max_timepoints, num_timepoints)

# Get the size of a single tile (assuming all images are of the same size)
first_image = next(iter(images_grid.values()))[0]  # Get the first frame from any stack
tile_width, tile_height = first_image.size

# Calculate the adjusted width and height considering the overlap
x_step = int(tile_width * (1 - overlap))
y_step = int(tile_height * (1 - overlap))

# Calculate canvas size accounting for overlap
stitched_width = x_step * (cols - 1) + tile_width
stitched_height = y_step * (rows - 1) + tile_height

# Iterate through the specified number of time points and save the stitched image separately
for t in range(timepoints_to_process):
    print(f"Processing time point {t}/{timepoints_to_process - 1}...")  # Debugging step
    
    # Create an empty canvas for the current time point
    stitched_image = Image.new("I", (stitched_width, stitched_height))  # Use "L" mode for grayscale
    
    # Paste each frame at time t into the correct position on the canvas
    for (x_index, y_index), image_stack in images_grid.items():
        # Reverse the x_index for correct horizontal ordering
        #reversed_x_index = cols - 1 - x_index
        
        # Get the specific frame (time point t) from the stack
        try:
            frame = image_stack[t]  # Correctly index by time point
            print(f"Frame for position {(x_index, y_index)} at time {t} successfully accessed.")
        except IndexError:
            print(f"Warning: Time point {t} out of range for stack at position {(x_index, y_index)}.")
            continue

        # Ensure the frame is in "L" mode for compatibility
        #if frame.mode != "L":
            #frame = frame.convert("L")

        # Calculate the position with overlap
        x_pos = x_index * x_step #reversed_
        y_pos = y_index * y_step
        
        # Paste the frame into the stitched image for this time point
        stitched_image.paste(frame, (x_pos, y_pos))
    
    # Save the stitched image for the current time point in the output directory
    output_filename = os.path.join(output_dir, f'{filename_prefix}_timepoint_{t}.tif')
    stitched_image.save(output_filename, compression="tiff_deflate")

    print(f"Saved stitched image for time point {t} to {output_filename}")


Processing time point 0/0...
Frame for position (0, 0) at time 0 successfully accessed.
Frame for position (1, 0) at time 0 successfully accessed.
Frame for position (2, 0) at time 0 successfully accessed.
Frame for position (3, 0) at time 0 successfully accessed.
Frame for position (4, 0) at time 0 successfully accessed.
Frame for position (4, 1) at time 0 successfully accessed.
Frame for position (3, 1) at time 0 successfully accessed.
Frame for position (2, 1) at time 0 successfully accessed.
Frame for position (1, 1) at time 0 successfully accessed.
Frame for position (0, 1) at time 0 successfully accessed.
Frame for position (0, 2) at time 0 successfully accessed.
Frame for position (1, 2) at time 0 successfully accessed.
Frame for position (2, 2) at time 0 successfully accessed.
Frame for position (3, 2) at time 0 successfully accessed.
Frame for position (4, 2) at time 0 successfully accessed.
Frame for position (4, 3) at time 0 successfully accessed.
Frame for position (3, 3) a